In [3]:
import pandas as pd
import numpy as np

In [4]:
df_materials = pd.read_csv("properties/materials.csv")

In [7]:
list(zip(df_materials["property"], df_materials["units"]))

[('dynamic viscosity', 'mPa.s'),
 ('dynamic viscosity', 'mPa.s'),
 ('dynamic viscosity', 'mPa.s'),
 ('dynamic viscosity', 'mPa.s'),
 ('density', 'kg/m3'),
 ('density', 'kg/m3'),
 ('density', 'kg/m3'),
 ('density', 'kg/m3'),
 ('surface tension', 'N/m'),
 ('surface tension', 'N/m'),
 ('surface tension', 'N/m'),
 ('surface tension', 'N/m'),
 ('density', 'kg/m3'),
 (nan, nan)]

In [33]:
df_materials.head()

,material,phase,temperature,property,value,units,source
0,MeOH,liquid,25.0,dynamic viscosity,0.538000,mPa.s,NaN
1,H2O,liquid,25.0,dynamic viscosity,0.913000,mPa.s,NaN
2,Sulfolane,liquid,25.0,dynamic viscosity,10.070000,mPa.s,NaN
3,THF,liquid,25.0,dynamic viscosity,0.456000,mPa.s,NaN
4,THF,liquid,25.0,density,879.960032,kg/m3,NaN


In [14]:
temp=25.0
selections = ['MeOH', 'H2O']
volumes = [100, 200]
all_liq_props = pd.DataFrame()

for n, liq in enumerate(selections):
    liq_props = df_materials[(df_materials["material"]==liq)&
                                   (df_materials["temperature"]==temp)].copy()
    liq_props['name'] = f"{liq}_{n+1}"
    
    liq_props = liq_props.pivot(index=['name'],columns=['property', 'units'], values='value').copy()
    liq_props[('volume', 'L')] = volumes[n]
    liq_props[('mass', 'kg')] = (liq_props[('density', 'kg/m3')]/1e3) * liq_props[('volume', 'L')]
    all_liq_props = pd.concat([all_liq_props, liq_props], axis=0)

# create dataframe of mass-weighted average properties
liq_props_avg = liq_props.copy()
total_volume = all_liq_props[('volume', 'L')].sum()
total_mass = all_liq_props[('mass', 'kg')].sum()
# liq_props_avg[('density', 'kg/m3')] = liq_props_avg[('mass', 'kg')] / (liq_props_avg[('volume', 'L')]/1e3)

all_liq_props['mass_fraction'] = all_liq_props[('mass', 'kg')] / total_mass

avg = all_liq_props.mul(all_liq_props[('mass_fraction')], axis=0).sum().to_frame().T
avg.index = ['mixture']
avg[('volume','L')] = total_volume
avg[('mass','kg')] = total_mass
avg[('mass_fraction')] = all_liq_props['mass_fraction'].sum()

all_liq_props = pd.concat([all_liq_props, avg], axis=0)
all_liq_props

property,dynamic viscosity,density,surface tension,volume,mass,mass_fraction
units,mPa.s,kg/m3,N/m,L,kg,
MeOH_1,0.538000,789.607467,0.022222,100,78.960747,0.283664
H2O_2,0.913000,996.997521,0.072118,200,199.399504,0.716336
mixture,0.806626,938.168455,0.057964,300,278.360251,1.000000


In [8]:
sm = {('name', ''): 'SM1',
                                          ('dynamic viscosity', 'mPa.s'): 1.0,
                                          ('density', 'kg/m3'): 10000.0,
                                          ('kinematic viscosity', 'm2/s'): 1.0e-6,
                                          ('surface tension', 'N/m'): 0.0728,
                                          ('volume', 'L'): 1.0,
                                          ('mass', 'kg'): 0.1
                                          }

In [15]:
sm = pd.DataFrame(sm, index=[0]).set_index(('name', ''))

In [16]:
pd.concat([sm, all_liq_props], axis=0)

,dynamic viscosity,density,kinematic viscosity,surface tension,volume,mass,mass_fraction
,mPa.s,kg/m3,m2/s,N/m,L,kg,
SM1,1.000000,10000.000000,0.000001,0.072800,1.0,0.100000,NaN
MeOH_1,0.538000,789.607467,NaN,0.022222,100.0,78.960747,0.283664
H2O_2,0.913000,996.997521,NaN,0.072118,200.0,199.399504,0.716336
mixture,0.806626,938.168455,NaN,0.057964,300.0,278.360251,1.000000
